In [1]:
import torch
import torch.nn as nn

from seisLM.data_pipeline import foreshock_aftershock_dataloaders as loaders
from seisLM.model.task_specific import foreshock_aftershock_models as models

dataloaders = loaders.prepare_foreshock_aftershock_dataloaders(
  num_classes=2,
  batch_size=32,
  component_order='ZNE',
  event_split_method='temporal'
)

Seed set to 42
Seed set to 42


In [2]:
# waveforms, batches = next(iter(dataloaders['train']))
# print(waveforms.shape)

In [3]:
from seisLM.model.foundation import pretrained_models
import json
import ml_collections
from seisLM.model.task_specific.foreshock_aftershock_models import Wav2Vec2ForSequenceClassification

config_path = '/scicore/home/dokman0000/liu0003/projects/seisLM/seisLM/configs/foreshock_aftershock/seisLM_shock_classifier.json'


with open(config_path, "r", encoding="utf-8") as f:
  model_config = json.load(f)
model_config = ml_collections.ConfigDict(model_config).model_args


pretrained_model = pretrained_models.LitMultiDimWav2Vec2.load_from_checkpoint(
  model_config.pretrained_ckpt_path
).model

## TODO: temp fix for the config issue

new_config = pretrained_model.config
for key, value in model_config.items():
  # new_config[key] = value
  setattr(new_config, key, value)

model_config = new_config
model = Wav2Vec2ForSequenceClassification(model_config)

model.wav2vec2.load_state_dict(
  pretrained_model.wav2vec2.state_dict()
)
model.freeze_feature_encoder()


In [4]:
batch = next(iter(dataloaders['train']))

waveforms, labels = batch
model(waveforms)

TypeError: forward() takes 1 positional argument but 2 were given

In [13]:
# model(input_values=waveforms)

# input_values = waveforms
# outputs = model.wav2vec2(
#     input_values,
#     attention_mask=None,
#     output_attentions=False,
#     output_hidden_states=False,
# )

# # if self.config.use_weighted_layer_sum:
# #     hidden_states = outputs[_HIDDEN_STATES_START_POSITION]
# #     hidden_states = torch.stack(hidden_states, dim=1)
# #     norm_weights = nn.functional.softmax(self.layer_weights, dim=-1)
# #     hidden_states = (hidden_states * norm_weights.view(-1, 1, 1)).sum(dim=1)
# # else:
# # TODO: implement the weighted layer sum version.

# hidden_states = outputs.last_hidden_state
# hidden_states = model.projector(hidden_states)
# pooled_output = hidden_states.mean(dim=1)
# logits = model.classifier(pooled_output)


tensor([[-9.3579e-03, -1.0114e-02, -3.6488e-02,  2.4164e-02, -1.7462e-02,
          1.1483e-02,  7.0830e-02, -5.0917e-03],
        [-3.7748e-02,  2.3717e-02, -3.4523e-02,  2.1795e-02,  2.7531e-02,
         -9.7431e-03,  6.2395e-02,  2.6684e-02],
        [-5.0361e-02, -1.2497e-02, -1.1258e-02,  1.2768e-02,  8.0464e-03,
          3.8091e-03,  6.9835e-02,  1.2279e-02],
        [-4.4289e-02, -3.7185e-02, -1.9467e-02,  3.1801e-02, -3.0180e-02,
          1.4837e-02,  3.5877e-02,  3.7556e-03],
        [-4.2501e-02, -1.4874e-02, -5.1696e-02,  1.4387e-02,  1.7219e-02,
          1.9677e-03,  3.0164e-02,  1.6754e-02],
        [-4.3103e-02,  6.9611e-02,  1.4611e-03, -4.2986e-03,  1.5814e-02,
          2.3448e-02,  1.0271e-01,  1.9547e-02],
        [-4.3008e-03, -1.9803e-02, -4.1481e-02,  2.9769e-02, -1.1264e-03,
         -3.7753e-03,  4.2563e-02, -1.4799e-02],
        [-3.8222e-03, -1.5987e-02, -3.6267e-02,  1.3079e-02,  1.5272e-03,
         -1.0030e-02,  2.9016e-02, -2.7978e-02],
        [ 6.4797

In [12]:
model

Wav2Vec2ForSequenceClassification(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(3, 256, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        )
        (1): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(256, 256, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=256, out_features=240, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): Wav2Vec2EncoderStableLayerNorm(
      (pos_conv_embed): Wav2Vec2PositionalConvEmbedding(
        (conv): ParametrizedConv1d(
          240, 240, kernel_size=(128,), stride=(1,), paddin

In [6]:
isinstance(wavforms, torch.Tensor)

True

In [7]:
# Example usage
# Input shape: (batch_size, 3, 2500) where 3 is the number of channels
# and 2500 is the time length
model = models.Conv1DShockClassifier(
  in_channels=3,
  num_classes=2,
  num_layers=5,
  initial_filters=32,
  kernel_size=3,
)



In [8]:
from torchinfo import summary

summary(model, input_size=(32, 3, 2500))

Layer (type:depth-idx)                   Output Shape              Param #
Conv1DShockClassifier                    [32, 2]                   --
├─Sequential: 1-1                        [32, 512, 75]             --
│    └─DoubleConvBlock: 2-1              [32, 32, 1248]            --
│    │    └─Sequential: 3-1              [32, 32, 1248]            3,488
│    └─DoubleConvBlock: 2-2              [32, 64, 622]             --
│    │    └─Sequential: 3-2              [32, 64, 622]             18,688
│    └─DoubleConvBlock: 2-3              [32, 128, 309]            --
│    │    └─Sequential: 3-3              [32, 128, 309]            74,240
│    └─DoubleConvBlock: 2-4              [32, 256, 153]            --
│    │    └─Sequential: 3-4              [32, 256, 153]            295,936
│    └─DoubleConvBlock: 2-5              [32, 512, 75]             --
│    │    └─Sequential: 3-5              [32, 512, 75]             1,181,696
├─AdaptiveAvgPool1d: 1-2                 [32, 512, 1]         

In [4]:
train_loader = dataloaders['train']

batch = next(iter(train_loader))

In [5]:
out = model(batch[0].cuda())

In [6]:
out.shape


torch.Size([32, 2])